# Test random varible evaluation

For first step we create import the sufficient packages 

In [1]:
import sys
import os
import time
sys.path.append('../')
from holoclean import holocleansession , dataset , dataengine
from holoclean.utils import domainpruning
from holoclean.featurization import dcfeaturizer,dc_featurizer

In [2]:
my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../holoclean/mysql-connector-java-5.1.44-bin.jar")
holoclean_se=holocleansession.HolocleanSession(path)

##  Creating Dataset and dataengine and holocleansession 
In this part we create dataset for keep tracking the runnig step and dataengine for make connector to the database we try to ingest the 10 element csv file and after ingesting it returns df other wise we can read it from database

In [3]:
ds=dataset.Dataset()  
d=dataengine.Dataengine("metadb-config.txt",'datadb-config.txt',ds,holoclean_se.return_sqlcontext())
holoclean_se.set_dataengine(d)
df=d.ingest_spark('10.csv',holoclean_se.returnspark_session())

index


##  Create featurizer


In [4]:
dcCode=['t1&t2&EQ(t1.city,t2.city)&EQ(t1.temp,t2.temp)&IQ(t1.tempType,t2.tempType)']
#dcCode=['t1&t2&EQ(t1.city,t2.city)&GTE(t1.temp,t2.temp)&IQ(t1.tempType,t2.tempType)']
dcf=dcfeaturizer.DCFeaturizer(df,dcCode,d)

In this version we create new table which ha sall possible value and by using join it with original table twe create the final table.

In [5]:
dc_f=dc_featurizer.DCFeaturizer(df,dcCode,d)

we claculate the creation time in this version :

In [6]:
start_time_dd = time.time()
dc_f.table_featurizer(d)
finish_time_dd=time.time()
mysqlv=d.get_table_spark("dc_f1")
print(mysqlv.count())
print(mysqlv.show())

136524702044_dc_f1
1884
+--------+-------+------------+----------------+------+---------+
|rv_index|rv_attr|assigned_val|         feature|  TYPE|weight_id|
+--------+-------+------------+----------------+------+---------+
|       1|   city| ITE00100554|   date=18000101|concur|        0|
|       1|   city| GM000010962|   date=18000101|concur|        0|
|       1|   city| EZE00100082|   date=18000101|concur|        0|
|       1|   city| ITE00100554|   date=18000102|concur|        1|
|       1|   city| GM000010962|   date=18000102|concur|        1|
|       1|   city| EZE00100082|   date=18000102|concur|        1|
|       1|   city| ITE00100554|INIT=ITE00100554|  init|        2|
|       1|   city| GM000010962|INIT=ITE00100554|  init|        2|
|       1|   city| EZE00100082|INIT=ITE00100554|  init|        2|
|       1|   city| ITE00100554|       temp=-125|concur|        3|
|       1|   city| GM000010962|       temp=-125|concur|        3|
|       1|   city| EZE00100082|       temp=-125|conc

In [7]:
print("--- %s seconds ---" % (finish_time_dd-start_time_dd))

--- 1.80709886551 seconds ---


In [8]:
start_time_dd = time.time()
dc_f.make_dc_f_table()
finish_time_dd=time.time()
mysqlv=d.get_table_spark("dc_f_dd")
print(mysqlv.count())
print(mysqlv.show())

24
+--------+--------+------------+--------------------+------+
|rv_index| rv_attr|assigned_val|                  DC|tup_id|
+--------+--------+------------+--------------------+------+
|      10|    city| EZE00100082|table1.city=table...|     5|
|       5|    city| EZE00100082|table1.city=table...|    10|
|       2|    temp|         -75|table1.city=table...|     1|
|       7|    temp|         -75|table1.city=table...|     1|
|       1|    temp|        -148|table1.city=table...|     2|
|       6|    temp|        -148|table1.city=table...|     2|
|      10|    temp|         -86|table1.city=table...|     4|
|      10|    temp|        -135|table1.city=table...|     5|
|       2|    temp|         -60|table1.city=table...|     6|
|       7|    temp|         -60|table1.city=table...|     6|
|       1|    temp|        -125|table1.city=table...|     7|
|       6|    temp|        -125|table1.city=table...|     7|
|      10|    temp|         -44|table1.city=table...|     9|
|       4|    temp|  

Execution time is :

In [9]:
print("--- %s seconds ---" % (finish_time_dd-start_time_dd))

--- 0.235793113708 seconds ---


Because we have just one dc in all value in dc column is same